# 3 Pointers Made against game_details.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis\3p
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [3]:
df = pd.read_csv('backend/data/details/game_details.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = shooting_df[['date', 'visitor', 'home', 'team', '3p']]

### Basic exploration

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506527 entries, 0 to 506526
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        506527 non-null  object 
 1   visitor     506527 non-null  object 
 2   home        506527 non-null  object 
 3   team        506527 non-null  int64  
 4   starter     506485 non-null  float64
 5   player      505841 non-null  object 
 6   mp          505841 non-null  object 
 7   fg          505841 non-null  float64
 8   fga         505841 non-null  float64
 9   fg_perc     484388 non-null  float64
 10  3p          505841 non-null  float64
 11  3pa         505841 non-null  float64
 12  3p_perc     354754 non-null  float64
 13  ft          505841 non-null  float64
 14  fta         505841 non-null  float64
 15  ft_perc     328284 non-null  float64
 16  orb         505841 non-null  float64
 17  drb         505841 non-null  float64
 18  trb         505841 non-null  float64
 19  as

In [5]:
df.sample(5)

,date,visitor,home,team,starter,player,mp,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus
28063,"Wed, Apr 11, 2007",Seattle SuperSonics,Phoenix Suns,0,0.0,Danny Fortson,0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74184,"Wed, Dec 31, 2008",Milwaukee Bucks,Houston Rockets,1,0.0,Von Wafer,0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261771,"Wed, Dec 17, 2014",Dallas Mavericks,Detroit Pistons,1,1.0,Kyle Singler,20:18,3.0,6.0,0.500,3.0,5.0,0.6,0.0,0.0,NaN,0.0,2.0,2.0,4.0,0.0,0.0,1.0,4.0,9.0,9.0
94562,"Wed, Oct 28, 2009",Indiana Pacers,Atlanta Hawks,1,0.0,Jamal Crawford,13:36,1.0,3.0,0.333,0.0,1.0,0.0,1.0,1.0,1.000,0.0,0.0,0.0,3.0,1.0,0.0,0.0,4.0,3.0,1.0
500921,"Sat, Feb 12, 2022",Oklahoma City Thunder,Chicago Bulls,0,0.0,Kenrich Williams,26:47,4.0,5.0,0.800,0.0,1.0,0.0,4.0,7.0,0.571,1.0,3.0,4.0,1.0,2.0,0.0,0.0,1.0,12.0,-6.0


In [6]:
def convert_mp(mp):
    if mp == '0' or mp == 0:
        return 0
    else:
        mins = int(mp.split(':')[0])
        secs = int(mp.split(':')[1]) / 60
        return mins + secs

In [7]:
# Fill NaN
df = df.fillna(0)

# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Convert 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Convert 'minutes played' to float
df['mp'] = df['mp'].apply(lambda x: convert_mp(x))

# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506527 entries, 0 to 506526
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        506527 non-null  datetime64[ns]
 1   visitor     506527 non-null  object        
 2   home        506527 non-null  object        
 3   team        506527 non-null  object        
 4   starter     506527 non-null  float64       
 5   player      506527 non-null  object        
 6   mp          506527 non-null  float64       
 7   fg          506527 non-null  float64       
 8   fga         506527 non-null  float64       
 9   fg_perc     506527 non-null  float64       
 10  3p          506527 non-null  float64       
 11  3pa         506527 non-null  float64       
 12  3p_perc     506527 non-null  float64       
 13  ft          506527 non-null  float64       
 14  fta         506527 non-null  float64       
 15  ft_perc     506527 non-null  float64       
 16  or

In [8]:
# Team total stats
teams_df = df.groupby(['date', 'visitor', 'home', 'team']).sum().reset_index()

In [9]:
# Rename target variable
shooting_df = shooting_df.rename({'3p': 'target'}, axis=1)

# Convert 'date' column to Date object
shooting_df['date'] = pd.to_datetime(shooting_df['date'])

# Convert 'team' column to Team Name
shooting_df['team'] = np.where(shooting_df['team'], shooting_df['home'], shooting_df['visitor'])

In [10]:
# Starters total stats
starters_df = df[df['starter'] == 1].groupby(['date', 'visitor', 'home', 'team']).sum()
starters_df = starters_df[stats]
starters_df = starters_df.reset_index()

# Merge dataframes to have target variable
starters_df = pd.merge(starters_df, shooting_df, 
                       left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
                       how='left')

In [11]:
# Bench total stats
bench_df = df[df['starter'] == 0].groupby(['date', 'visitor', 'home', 'team']).sum()
bench_df = bench_df[stats]
bench_df = bench_df.reset_index()

# Merge dataframes to have target variable
bench_df = pd.merge(bench_df, shooting_df, 
                    left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
                    how='left')

# Dataframe of team's last 15 performances

In [12]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = teams_df[teams_df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

teams_df['dates'] = teams_df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
teams_df['date_1'], teams_df['date_2'] = teams_df['dates'].apply(lambda x: x[0]), teams_df['dates'].apply(lambda x: x[1])
teams_df['date_3'], teams_df['date_4'] = teams_df['dates'].apply(lambda x: x[2]), teams_df['dates'].apply(lambda x: x[3])
teams_df['date_5'], teams_df['date_6'] = teams_df['dates'].apply(lambda x: x[4]), teams_df['dates'].apply(lambda x: x[5])
teams_df['date_7'], teams_df['date_8'] = teams_df['dates'].apply(lambda x: x[6]), teams_df['dates'].apply(lambda x: x[7])
teams_df['date_9'], teams_df['date_10'] = teams_df['dates'].apply(lambda x: x[8]), teams_df['dates'].apply(lambda x: x[9])
teams_df['date_11'], teams_df['date_12'] = teams_df['dates'].apply(lambda x: x[10]), teams_df['dates'].apply(lambda x: x[11])
teams_df['date_13'], teams_df['date_14'] = teams_df['dates'].apply(lambda x: x[12]), teams_df['dates'].apply(lambda x: x[13])
teams_df['date_15'] = teams_df['dates'].apply(lambda x: x[14])

In [13]:
# Merge in opponents (see team defensive stats)
starters_df = pd.merge(
    starters_df, 
    starters_df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

starters_df = starters_df[starters_df['team'] != starters_df['team_opp']]

bench_df = pd.merge(
    bench_df, 
    bench_df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

bench_df = bench_df[bench_df['team'] != bench_df['team_opp']]

In [14]:
# Keep date columns in teams
cols = [col for col in teams_df.columns
        if ('date_' in col) or \
        (col in ['date', 'visitor', 'home', 'team'])]
teams_df = teams_df[cols]

# Merge dates with starters
starters_df = pd.merge(starters_df, teams_df, 
                       left_on=['date', 'visitor', 'home', 'team'], 
                       right_on=['date', 'visitor', 'home', 'team'],
                       how='left')

# Merge dates with bench
bench_df = pd.merge(bench_df, teams_df, 
                    left_on=['date', 'visitor', 'home', 'team'], 
                    right_on=['date', 'visitor', 'home', 'team'],
                    how='left')

In [15]:
starters_df.head()

,date,visitor,home,team,fg,fga,3p,3pa,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,mp,target,team_opp,fg_opp,fga_opp,3p_opp,3pa_opp,ft_opp,fta_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,plus_minus_opp,mp_opp,target_opp,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15
0,2006-10-31,Chicago Bulls,Miami Heat,Chicago Bulls,18.0,43.0,3.0,7.0,14.0,18.0,7.0,14.0,21.0,8.0,5.0,2.0,7.0,12.0,53.0,66.0,128.550000,7.0,Miami Heat,20.0,48.0,3.0,13.0,11.0,19.0,4.0,18.0,22.0,9.0,4.0,3.0,14.0,14.0,54.0,-95.0,156.583333,3.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2006-10-31,Chicago Bulls,Miami Heat,Miami Heat,20.0,48.0,3.0,13.0,11.0,19.0,4.0,18.0,22.0,9.0,4.0,3.0,14.0,14.0,54.0,-95.0,156.583333,3.0,Chicago Bulls,18.0,43.0,3.0,7.0,14.0,18.0,7.0,14.0,21.0,8.0,5.0,2.0,7.0,12.0,53.0,66.0,128.550000,7.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2006-10-31,Phoenix Suns,Los Angeles Lakers,Los Angeles Lakers,26.0,51.0,5.0,10.0,15.0,20.0,7.0,26.0,33.0,24.0,7.0,0.0,13.0,10.0,72.0,-5.0,157.600000,6.0,Phoenix Suns,24.0,49.0,6.0,18.0,5.0,5.0,4.0,20.0,24.0,22.0,1.0,4.0,11.0,13.0,59.0,-11.0,159.483333,13.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2006-10-31,Phoenix Suns,Los Angeles Lakers,Phoenix Suns,24.0,49.0,6.0,18.0,5.0,5.0,4.0,20.0,24.0,22.0,1.0,4.0,11.0,13.0,59.0,-11.0,159.483333,13.0,Los Angeles Lakers,26.0,51.0,5.0,10.0,15.0,20.0,7.0,26.0,33.0,24.0,7.0,0.0,13.0,10.0,72.0,-5.0,157.600000,6.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,2006-11-01,Atlanta Hawks,Philadelphia 76ers,Atlanta Hawks,15.0,49.0,3.0,12.0,15.0,18.0,8.0,19.0,27.0,7.0,7.0,4.0,15.0,16.0,48.0,-83.0,164.683333,4.0,Philadelphia 76ers,24.0,56.0,3.0,5.0,21.0,23.0,13.0,21.0,34.0,17.0,7.0,5.0,14.0,12.0,72.0,78.0,168.800000,3.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


# Starters Analysis

In [17]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = starters_df.copy()
X = starters_df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

### Last 15 Performances (Unweighted)

In [18]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']

last_15 = last_15_games.copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat + '_last_15'] = 0
    last_15[stat + '_last_3'] = 0
    last_15[stat + '_last_1'] = 0
    
    for date in dates:
        # Last 15 games
        last_15[stat + '_last_15'] = last_15[stat + '_last_15'] + last_15[stat + date]
        
        # Last 3 games
        if date in ['_1', '_2', '_3']:
            last_15[stat + '_last_3'] = last_15[stat + '_last_3'] + last_15[stat + date]
        
        # Last game
        if date in ['_1']:
            last_15[stat + '_last_1'] = last_15[stat + '_last_1'] + last_15[stat + date]
        
    
    last_15[stat + '_last_15'] = last_15[stat + '_last_15'] / 15
    last_15[stat + '_last_3'] = last_15[stat + '_last_3'] / 3

# Calculate difference between last 15 games, 3 games and last game
for stat in stats:
    last_15[stat + '_trend_3'] = last_15[stat + '_last_15'] - last_15[stat + '_last_3']
    last_15[stat + '_trend_1'] = last_15[stat + '_last_15'] - last_15[stat + '_last_1']

# Sum stats for opposing teams for each game
last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Keep columns
stats_15 = [stat + '_last_15' for stat in stats]
stats_3 = [stat + '_last_3' for stat in stats]
stats_1 = [stat + '_last_1' for stat in stats]
trend_3 = [stat + '_trend_3' for stat in stats]
trend_1 = [stat + '_trend_1' for stat in stats]
last_15 = last_15[['target'] + stats_1 + stats_3 + stats_15 + trend_1 + trend_3]

# Calculate percentages
last_15['fg_perc_last_15'] = last_15['fg_last_15'] / last_15['fga_last_15']
last_15['fg_perc_opp_last_15'] = last_15['fg_opp_last_15'] / last_15['fga_opp_last_15']

last_15['3p_perc_last_15'] = last_15['3p_last_15'] / last_15['3pa_last_15']
last_15['3p_perc_opp_last_15'] = last_15['3p_opp_last_15'] / last_15['3pa_opp_last_15']

last_15['ft_perc_last_15'] = last_15['ft_last_15'] / last_15['fta_last_15']
last_15['ft_perc_opp_last_15'] = last_15['ft_opp_last_15'] / last_15['fta_opp_last_15']

# Calculate advanced stats
last_15['ts_perc_last_15'] = last_15['pts_last_15'] / (2 * (last_15['fga_last_15'] + .44 * last_15['fta_last_15']))
last_15['ts_perc_opp_last_15'] = last_15['pts_opp_last_15'] / (2 * (last_15['fga_opp_last_15'] + .44 * last_15['fta_opp_last_15']))

last_15['efg_perc_last_15'] = (last_15['fg_last_15'] + (.5 * last_15['3p_last_15'])) / last_15['fga_last_15']
last_15['efg_perc_opp_last_15'] = (last_15['fg_opp_last_15'] + (.5 * last_15['3p_opp_last_15'])) / last_15['fga_opp_last_15']

last_15['3par_last_15'] = last_15['3pa_last_15'] / last_15['fga_last_15']
last_15['3par_opp_last_15'] = last_15['3pa_opp_last_15'] / last_15['fga_opp_last_15']

last_15['ftr_last_15'] = last_15['fta_last_15'] / last_15['fga_last_15']
last_15['ftr_opp_last_15'] = last_15['fta_opp_last_15'] / last_15['fga_opp_last_15']

last_15['orb_perc_last_15'] = last_15['orb_last_15'] / (last_15['orb_last_15'] + last_15['drb_opp_last_15'])
last_15['orb_perc_opp_last_15'] = last_15['orb_opp_last_15'] / (last_15['orb_opp_last_15'] + last_15['drb_last_15'])

last_15['drb_perc_last_15'] = last_15['drb_last_15'] / (last_15['drb_last_15'] + last_15['orb_opp_last_15'])
last_15['drb_perc_opp_last_15'] = last_15['drb_opp_last_15'] / (last_15['drb_opp_last_15'] + last_15['orb_last_15'])

last_15['trb_perc_last_15'] = last_15['trb_last_15'] / (last_15['trb_last_15'] + last_15['trb_opp_last_15'])
last_15['trb_perc_opp_last_15'] = last_15['trb_opp_last_15'] / (last_15['trb_opp_last_15'] + last_15['trb_last_15'])

last_15['ast_perc_last_15'] = last_15['ast_last_15'] / last_15['fg_last_15']
last_15['ast_perc_opp_last_15'] = last_15['ast_opp_last_15'] / last_15['fg_opp_last_15']

starters_15_games = last_15.dropna(axis=0).copy()
starters_15_games.tail()

target  fg_last_1  \
date       visitor                home                                      
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers     21.0       64.0   
           Phoenix Suns           Houston Rockets         29.0       52.0   
           Portland Trail Blazers New York Knicks         27.0       67.0   
           Toronto Raptors        Los Angeles Clippers    22.0       64.0   
2022-03-17 Detroit Pistons        Orlando Magic            0.0       39.0   

                                                        fga_last_1  3p_last_1  \
date       visitor                home                                          
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers        123.0       20.0   
           Phoenix Suns           Houston Rockets            101.0       19.0   
           Portland Trail Blazers New York Knicks            133.0       17.0   
           Toronto Raptors        Los Angeles Clippers       156.0       17.0   
2022-03-17 Detroit Pistons        Orlando Magic               98.0       14.0   

                                                        3pa_last_1  ft_last_1  \
date       visitor                home                                          
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         40.0       44.0   
           Phoenix Suns           Houston Rockets             42.0       17.0   
           Portland Trail Blazers New York Knicks             40.0       31.0   
           Toronto Raptors        Los Angeles Clippers        59.0       33.0   
2022-03-17 Detroit Pistons        Orlando Magic               40.0       29.0   

                                                        fta_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         55.0   
           Phoenix Suns           Houston Rockets             21.0   
           Portland Trail Blazers New York Knicks             39.0   
           Toronto Raptors        Los Angeles Clippers        43.0   
2022-03-17 Detroit Pistons        Orlando Magic               38.0   

                                                        orb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         11.0   
           Phoenix Suns           Houston Rockets              8.0   
           Portland Trail Blazers New York Knicks             16.0   
           Toronto Raptors        Los Angeles Clippers        21.0   
2022-03-17 Detroit Pistons        Orlando Magic                6.0   

                                                        drb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         48.0   
           Phoenix Suns           Houston Rockets             33.0   
           Portland Trail Blazers New York Knicks             44.0   
           Toronto Raptors        Los Angeles Clippers        55.0   
2022-03-17 Detroit Pistons        Orlando Magic               30.0   

                                                        trb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         59.0   
           Phoenix Suns           Houston Rockets             41.0   
           Portland Trail Blazers New York Knicks             60.0   
           Toronto Raptors        Los Angeles Clippers        76.0   
2022-03-17 Detroit Pistons        Orlando Magic               36.0   

                                                        ast_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         43.0   
           Phoenix Suns           Houston Rockets             38.0   
           Portland Trail Blazers New York Knicks             38.0   
           Toronto Raptors        Los Angeles Clippers        42.0

## Correlations

In [19]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in starters_15_games:
    corr_p = pearsonr(starters_15_games['target'], starters_15_games[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print statistically significant correlations
starters_corr = corr_df[corr_df['p-value'] < .05].sort_values(['corr'], axis=0, ascending=False)
starters_corr

,corr,p-value,stat
0,1.00,0.0,target
72,0.71,0.0,3pa_last_15
181,0.71,0.0,3par_last_15
71,0.70,0.0,3p_last_15
182,0.68,0.0,3par_opp_last_15
...,...,...,...
92,-0.29,0.0,orb_opp_last_15
183,-0.30,0.0,ftr_last_15
184,-0.37,0.0,ftr_opp_last_15
185,-0.38,0.0,orb_perc_last_15


## Bench Analysis

In [20]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = bench_df.copy()
X = bench_df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

### Last 15 Performances (Unweighted)

In [21]:
# Set stats
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 
         'tov', 'pf', 'pts', 'plus_minus', 'mp']
opp_stats = [stat + '_opp' for stat in stats]
stats = stats + opp_stats

dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']

last_15 = last_15_games.copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat + '_last_15'] = 0
    last_15[stat + '_last_3'] = 0
    last_15[stat + '_last_1'] = 0
    
    for date in dates:
        # Last 15 games
        last_15[stat + '_last_15'] = last_15[stat + '_last_15'] + last_15[stat + date]
        
        # Last 3 games
        if date in ['_1', '_2', '_3']:
            last_15[stat + '_last_3'] = last_15[stat + '_last_3'] + last_15[stat + date]
        
        # Last game
        if date in ['_1']:
            last_15[stat + '_last_1'] = last_15[stat + '_last_1'] + last_15[stat + date]
        
    
    last_15[stat + '_last_15'] = last_15[stat + '_last_15'] / 15
    last_15[stat + '_last_3'] = last_15[stat + '_last_3'] / 3

# Calculate difference between last 15 games, 3 games and last game
for stat in stats:
    last_15[stat + '_trend_3'] = last_15[stat + '_last_15'] - last_15[stat + '_last_3']
    last_15[stat + '_trend_1'] = last_15[stat + '_last_15'] - last_15[stat + '_last_1']

# Sum stats for opposing teams for each game
last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Keep columns
stats_15 = [stat + '_last_15' for stat in stats]
stats_3 = [stat + '_last_3' for stat in stats]
stats_1 = [stat + '_last_1' for stat in stats]
trend_3 = [stat + '_trend_3' for stat in stats]
trend_1 = [stat + '_trend_1' for stat in stats]
last_15 = last_15[['target'] + stats_1 + stats_3 + stats_15 + trend_1 + trend_3]

# Calculate percentages
last_15['fg_perc_last_15'] = last_15['fg_last_15'] / last_15['fga_last_15']
last_15['fg_perc_opp_last_15'] = last_15['fg_opp_last_15'] / last_15['fga_opp_last_15']

last_15['3p_perc_last_15'] = last_15['3p_last_15'] / last_15['3pa_last_15']
last_15['3p_perc_opp_last_15'] = last_15['3p_opp_last_15'] / last_15['3pa_opp_last_15']

last_15['ft_perc_last_15'] = last_15['ft_last_15'] / last_15['fta_last_15']
last_15['ft_perc_opp_last_15'] = last_15['ft_opp_last_15'] / last_15['fta_opp_last_15']

# Calculate advanced stats
last_15['ts_perc_last_15'] = last_15['pts_last_15'] / (2 * (last_15['fga_last_15'] + .44 * last_15['fta_last_15']))
last_15['ts_perc_opp_last_15'] = last_15['pts_opp_last_15'] / (2 * (last_15['fga_opp_last_15'] + .44 * last_15['fta_opp_last_15']))

last_15['efg_perc_last_15'] = (last_15['fg_last_15'] + (.5 * last_15['3p_last_15'])) / last_15['fga_last_15']
last_15['efg_perc_opp_last_15'] = (last_15['fg_opp_last_15'] + (.5 * last_15['3p_opp_last_15'])) / last_15['fga_opp_last_15']

last_15['3par_last_15'] = last_15['3pa_last_15'] / last_15['fga_last_15']
last_15['3par_opp_last_15'] = last_15['3pa_opp_last_15'] / last_15['fga_opp_last_15']

last_15['ftr_last_15'] = last_15['fta_last_15'] / last_15['fga_last_15']
last_15['ftr_opp_last_15'] = last_15['fta_opp_last_15'] / last_15['fga_opp_last_15']

last_15['orb_perc_last_15'] = last_15['orb_last_15'] / (last_15['orb_last_15'] + last_15['drb_opp_last_15'])
last_15['orb_perc_opp_last_15'] = last_15['orb_opp_last_15'] / (last_15['orb_opp_last_15'] + last_15['drb_last_15'])

last_15['drb_perc_last_15'] = last_15['drb_last_15'] / (last_15['drb_last_15'] + last_15['orb_opp_last_15'])
last_15['drb_perc_opp_last_15'] = last_15['drb_opp_last_15'] / (last_15['drb_opp_last_15'] + last_15['orb_last_15'])

last_15['trb_perc_last_15'] = last_15['trb_last_15'] / (last_15['trb_last_15'] + last_15['trb_opp_last_15'])
last_15['trb_perc_opp_last_15'] = last_15['trb_opp_last_15'] / (last_15['trb_opp_last_15'] + last_15['trb_last_15'])

last_15['ast_perc_last_15'] = last_15['ast_last_15'] / last_15['fg_last_15']
last_15['ast_perc_opp_last_15'] = last_15['ast_opp_last_15'] / last_15['fg_opp_last_15']

bench_15_games = last_15.dropna(axis=0).copy()
bench_15_games.tail()

target  fg_last_1  \
date       visitor                home                                      
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers     21.0       13.0   
           Phoenix Suns           Houston Rockets         29.0       36.0   
           Portland Trail Blazers New York Knicks         27.0       16.0   
           Toronto Raptors        Los Angeles Clippers    22.0       18.0   
2022-03-17 Detroit Pistons        Orlando Magic            0.0       34.0   

                                                        fga_last_1  3p_last_1  \
date       visitor                home                                          
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         47.0        7.0   
           Phoenix Suns           Houston Rockets             66.0       11.0   
           Portland Trail Blazers New York Knicks             38.0        4.0   
           Toronto Raptors        Los Angeles Clippers        38.0        5.0   
2022-03-17 Detroit Pistons        Orlando Magic               77.0       11.0   

                                                        3pa_last_1  ft_last_1  \
date       visitor                home                                          
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         23.0        5.0   
           Phoenix Suns           Houston Rockets             23.0       13.0   
           Portland Trail Blazers New York Knicks             13.0        2.0   
           Toronto Raptors        Los Angeles Clippers        14.0        6.0   
2022-03-17 Detroit Pistons        Orlando Magic               35.0        6.0   

                                                        fta_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers          9.0   
           Phoenix Suns           Houston Rockets             19.0   
           Portland Trail Blazers New York Knicks              3.0   
           Toronto Raptors        Los Angeles Clippers        10.0   
2022-03-17 Detroit Pistons        Orlando Magic               11.0   

                                                        orb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         12.0   
           Phoenix Suns           Houston Rockets              7.0   
           Portland Trail Blazers New York Knicks              6.0   
           Toronto Raptors        Los Angeles Clippers        10.0   
2022-03-17 Detroit Pistons        Orlando Magic               11.0   

                                                        drb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         26.0   
           Phoenix Suns           Houston Rockets             25.0   
           Portland Trail Blazers New York Knicks             30.0   
           Toronto Raptors        Los Angeles Clippers        21.0   
2022-03-17 Detroit Pistons        Orlando Magic               28.0   

                                                        trb_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         38.0   
           Phoenix Suns           Houston Rockets             32.0   
           Portland Trail Blazers New York Knicks             36.0   
           Toronto Raptors        Los Angeles Clippers        31.0   
2022-03-17 Detroit Pistons        Orlando Magic               39.0   

                                                        ast_last_1  \
date       visitor                home                               
2022-03-16 Philadelphia 76ers     Cleveland Cavaliers         10.0   
           Phoenix Suns           Houston Rockets             21.0   
           Portland Trail Blazers New York Knicks             14.0   
           Toronto Raptors        Los Angeles Clippers        11.0

## Correlations of Bench

In [22]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in bench_15_games:
    corr_p = pearsonr(bench_15_games['target'], bench_15_games[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print statistically significant correlations
bench_corr = corr_df[corr_df['p-value'] < .05].sort_values(['corr'], axis=0, ascending=False)
bench_corr

,corr,p-value,stat
0,1.00,0.0,target
89,0.66,0.0,3pa_opp_last_15
182,0.65,0.0,3par_opp_last_15
72,0.63,0.0,3pa_last_15
88,0.62,0.0,3p_opp_last_15
...,...,...,...
74,-0.09,0.0,fta_last_15
186,-0.29,0.0,orb_perc_opp_last_15
183,-0.34,0.0,ftr_last_15
185,-0.35,0.0,orb_perc_last_15


# Comparison of Starters to Bench

In [23]:
corr_df = pd.merge(starters_corr.drop(['p-value'], axis=1), 
                   bench_corr.drop(['p-value'], axis=1),
                   left_on=['stat'], right_on=['stat'],
                   how='outer',
                   suffixes=['_starter', '_bench'])
corr_df.sort_values(['stat'], axis=0)

,corr_starter,stat,corr_bench
12,0.53,3p_last_1,0.37
3,0.70,3p_last_15,0.60
9,0.63,3p_last_3,0.49
15,0.44,3p_opp_last_1,0.33
7,0.66,3p_opp_last_15,0.62
...,...,...,...
58,0.10,trb_opp_last_3,0.28
75,-0.02,trb_perc_last_15,-0.05
68,0.02,trb_perc_opp_last_15,0.05
17,0.41,ts_perc_last_15,0.36


## Save dataframe with significantly correlated stats

In [24]:
correlation = .6

starter_stats = starters_corr[starters_corr['corr'].abs() >= correlation]['stat']
starters_df = starters_15_games[starter_stats]

bench_stats = bench_corr[bench_corr['corr'].abs() >= correlation]['stat']
bench_df = bench_15_games[bench_stats]

df = pd.merge(starters_df, bench_df, 
              left_on=['date', 'visitor', 'home'], 
              right_on=['date', 'visitor', 'home'], 
              how='outer', suffixes=['_starters', '_bench'])

df = df.drop(['target_bench', 'target_starters'], axis=1)

df.head()

df.to_csv('backend/data/inputs/3p/game_details.csv')

3pa_last_15_starters  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers              7.866667   
           New York Knicks Chicago Bulls                       8.400000   
2006-11-29 Indiana Pacers  Golden State Warriors              25.933333   
           New York Knicks Cleveland Cavaliers                 7.933333   
           Orlando Magic   Seattle SuperSonics                22.800000   

                                                   3par_last_15_starters  \
date       visitor         home                                            
2006-11-28 Indiana Pacers  Portland Trail Blazers               0.173785   
           New York Knicks Chicago Bulls                        0.173793   
2006-11-29 Indiana Pacers  Golden State Warriors                0.230724   
           New York Knicks Cleveland Cavaliers                  0.164592   
           Orlando Magic   Seattle SuperSonics                  0.210981   

                                                   3p_last_15_starters  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers             2.466667   
           New York Knicks Chicago Bulls                      3.066667   
2006-11-29 Indiana Pacers  Golden State Warriors              9.066667   
           New York Knicks Cleveland Cavaliers                2.733333   
           Orlando Magic   Seattle SuperSonics                8.400000   

                                                   3par_opp_last_15_starters  \
date       visitor         home                                                
2006-11-28 Indiana Pacers  Portland Trail Blazers                   0.206725   
           New York Knicks Chicago Bulls                            0.204517   
2006-11-29 Indiana Pacers  Golden State Warriors                    0.176871   
           New York Knicks Cleveland Cavaliers                      0.200000   
           Orlando Magic   Seattle SuperSonics                      0.179187   

                                                   3pa_opp_last_15_starters  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers                 11.066667   
           New York Knicks Chicago Bulls                          10.866667   
2006-11-29 Indiana Pacers  Golden State Warriors                  19.066667   
           New York Knicks Cleveland Cavaliers                    10.600000   
           Orlando Magic   Seattle SuperSonics                    19.400000   

                                                   3pa_last_3  \
date       visitor         home                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers    8.000000   
           New York Knicks Chicago Bulls             4.666667   
2006-11-29 Indiana Pacers  Golden State Warriors    25.000000   
           New York Knicks Cleveland Cavaliers       4.000000   
           Orlando Magic   Seattle SuperSonics      21.666667   

                                                   3p_opp_last_15_starters  \
date       visitor         home                                              
2006-11-28 Indiana Pacers  Portland Trail Blazers                 4.466667   
           New York Knicks Chicago Bulls                          3.533333   
2006-11-29 Indiana Pacers  Golden State Warriors                  6.533333   
           New York Knicks Cleveland Cavaliers                    3.533333   
           Orlando Magic   Seattle SuperSonics                    7.000000   

                                                   3pa_opp_last_3  3p_last_3  \
date       visitor         home                                                
2006-11-28 Indiana Pacers  Portland Trail Blazers       17.333333   3.666667   
           New York Knicks Chicago Bulls                10.666667   1.333333   
2006-11-29 Indiana Pacers  Golden S